# HW 2: Corporate Bond Pricing (due by 9.21 Fri)

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2018`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2018/HW2/HW2.ipynb`__  (Please use the same name for repository and ipynb file)
* Adding more code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [8]:
import numpy as np

In [9]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [10]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 1000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)

(9.595696163062215, 10.0)

In [12]:
# Put your code here to price the corporate bond

def corp_bond(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1000):  
    U = np.random.uniform(size=n_sample)
    default_time = -(1/def_rate)*np.log(U)

    price = np.zeros(n_sample)
    
    for i in range(n_sample):
        
        if default_time[i] > mat:
            price[i] = np.exp(-rf_rate*mat)
        else:
            price[i] = recovery*np.exp(-rf_rate*default_time[i])
            
    return np.mean(price)

# Call your function
corp_bond(mat, def_rate, rf_rate, recovery)




0.4364757807333766

In [13]:
# Find the mean and std by calling the function 100 times. 

price = np.zeros(100)

for i in range(100): 
    price [i] = corp_bond(mat, def_rate, rf_rate, recovery)
    price_100 = np.mean(price)
    std = np.std(price)
print(price_100, std)

0.4404079774113997 0.0067743968819815766


### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [14]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(10.000000000000002, 10.0)

In [17]:
# No include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1000, antithetic=True, mean_match=True):
    
    U = np.random.uniform(size=n_sample)
    
    if(antithetic):
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
    if(mean_match):
        default_time = -(1/def_rate)*np.log(U)
        default_time += 1/def_rate-default_time.mean() 
    else:
        default_time = -(1/def_rate)*np.log(U)
    
    price = np.zeros(n_sample)

    for i in range(n_sample):
        if default_time[i] > mat:
            price[i] = np.exp(-rf_rate*mat)
        else:
            price[i] = recovery*np.exp(-rf_rate*default_time[i])
            
    return np.mean(price)

corp_bond_cv(mat, def_rate, rf_rate, recovery)

0.4429280609832291

In [18]:
# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both

#ANTITHETIC 

price = np.zeros(100)

for i in range(100): 
    price [i] = corp_bond_cv(mat, def_rate, rf_rate, recovery, antithetic=True, mean_match=False)
    price_100 = np.mean(price)
    std = np.std(price)
print(price_100, std)

0.4396301452027593 0.006934258758601964


In [19]:
#MEAN MATCH

price = np.zeros(100)

for i in range(100): 
    price [i] = corp_bond_cv(mat, def_rate, rf_rate, recovery, antithetic=False, mean_match=True)
    price_100 = np.mean(price)
    std = np.std(price)
print(price_100, std)

0.44066191803436405 0.004360105717674882


In [20]:
#BOTH

price = np.zeros(100)

for i in range(100): 
    price [i] = corp_bond_cv(mat, def_rate, rf_rate, recovery, antithetic=True, mean_match=True)
    price_100 = np.mean(price)
    std = np.std(price)
print(price_100, std)

0.4405808754433824 0.005074298292399267


### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?

In [13]:
### Put the analytic expression for the corporate bond price

#Analytic = Integral from 0 to maturity {recovery*exp(-rf_rate*default_time)*def_rate*exp(-def_rate*default_time)*default_time}+ exp(-rf_rate*mat)*exp(-def_rate*mat)
#Default time changes to maturity since analytic expressions do not use default times

In [9]:
from math import cos, exp, pi
from scipy.integrate import quad
import numpy as np

def corp_bond(Z):
    def_rate = 0.1
    rf_rate = 0.03
    recovery = 0.3
    mat = 10
    return recovery*exp(-rf_rate*Z)*def_rate*exp(-def_rate*Z)


In [13]:
mat = 10
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
res = quad(corp_bond,0,mat)
print(res)

(0.16787727853061246, 1.863812199360964e-15)


In [16]:
res1 = 0.16787727853061246
price = exp(-rf_rate*mat)*exp(-def_rate*mat) + res1
print(price)

0.44040907156462505


By reducing the MC variations, we have lowered our standard deviation in comparison to the analytic value. Although the answers are similar, the analytic expression will reveal a greater value (meaning more expensive) than that of a lower standard deviation. This would mean that the corporate bond is cheaper once the variance is lowered.